# Data Preparation

The purpose of this notebook is only to pull from the original json file and export to csv. The intent is to make it easier to do subsequent exploration, visualization, and modeling.

In [1]:
import json
import pandas as pd

In [2]:
reviews = []
for line in open('json/dev/dataset_en_dev.json', 'r'):
    reviews.append(json.loads(line))

In [3]:
len(reviews)

5000

In [4]:
reviews[1]['stars']

'1'

In [5]:
reviews[1]['review_title']

'Garbage!'

In [6]:
reviews[1]['product_category']

'shoes'

In [7]:
reviews[1]['review_body']

"Shoes were purchased on March 6, 2019. My wife wore them a couple times a week since receiving them. April 19, 2019 one of the leather straps broke. Checked return policy and it expired on April 6. One month of warranty? Won't buy shoes from Amazon again."

In [8]:
reviews[1]

{'review_id': 'en_0830781',
 'product_id': 'product_en_0004522',
 'reviewer_id': 'reviewer_en_0731158',
 'stars': '1',
 'review_body': "Shoes were purchased on March 6, 2019. My wife wore them a couple times a week since receiving them. April 19, 2019 one of the leather straps broke. Checked return policy and it expired on April 6. One month of warranty? Won't buy shoes from Amazon again.",
 'review_title': 'Garbage!',
 'language': 'en',
 'product_category': 'shoes'}

In [9]:
# Create individual data columns
def extract_feature(label):
    output = []
    for i in range(0,len(reviews)):
        output.append(reviews[i][label])
    return(pd.DataFrame(output))

# stars = []
# for i in range(0,len(reviews)):
#     stars.append(reviews[i]['stars'])

In [10]:
stars = extract_feature('stars')
title = extract_feature('review_title')
category = extract_feature('product_category')
body = extract_feature('review_body')
reviewer = extract_feature('reviewer_id')

In [11]:
stars.value_counts()

1    1000
2    1000
3    1000
4    1000
5    1000
dtype: int64

Interesting, perfectly balanced category counts...however, stars might be more cast to integer values:

Is there a possiblity that we have multiple reviews by the same reviewer? If so that might be something to consider, e.g. if there's evidence they're machine generated they might be removed; or if there are many multi-review submissions we can normalize to that reviewer (is it someone who tends to give mostly 1's or mostly 5's?)

In [12]:
reviewer.columns = ['id']

In [13]:
# General form of id itself: remove the leading 'reviewer_en_'
reviewer_id = reviewer['id'].apply(lambda x: x.split('_')[-1])

In [14]:
reviewer_id.value_counts()

0228364    2
0555851    2
0987470    1
0786257    1
0366796    1
          ..
0039415    1
0888331    1
0571450    1
0008837    1
0850986    1
Name: id, Length: 4998, dtype: int64

Meh, only two reviewers in the whole set submitted two reviews each. 

In [15]:
df = pd.concat([stars, category, title, body], axis=1)
df.columns = ['stars','category','title','body']

In [16]:
df.head()

,stars,category,title,body
0,1,baby_product,Not worth the price and very bad cap design,Pathetic design of the caps. Very impractical ...
1,1,shoes,Garbage!,"Shoes were purchased on March 6, 2019. My wife..."
2,1,office_product,I do not recommend this printer,It's taken me 1 whole year to set this thing u...
3,1,office_product,Don't purchase these refurbished cartridges!,Each cartridge printed once. Both dried up in ...
4,1,baby_product,Not worth,No light hard to see


In [17]:
df.to_csv('csv/dataset_en_dev.csv', index=False)

# Now to do the (large) train file:


In [18]:
reviews = []
for line in open('json/train/dataset_en_train.json', 'r'):
    reviews.append(json.loads(line))

In [19]:
stars = extract_feature('stars')
title = extract_feature('review_title')
category = extract_feature('product_category')
body = extract_feature('review_body')

In [20]:
df = pd.concat([stars, category, title, body], axis=1)
df.columns = ['stars','category','title','body']

In [21]:
df.describe()

,stars,category,title,body
count,200000,200000,200000,200000
unique,5,31,137406,199426
top,1,home,Three Stars,Smaller than expected
freq,40000,17679,4169,29


In [22]:
df.to_csv('csv/dataset_en_train.csv', index=False)

# Next the smaller test file:

In [23]:
reviews = []
for line in open('json/test/dataset_en_test.json', 'r'):
    reviews.append(json.loads(line))
    

In [24]:
stars = extract_feature('stars')
title = extract_feature('review_title')
category = extract_feature('product_category')
body = extract_feature('review_body')

In [25]:
df = pd.concat([stars, category, title, body], axis=1)
df.columns = ['stars','category','title','body']

In [26]:
df.describe()

,stars,category,title,body
count,5000,5000,5000,5000
unique,5,31,4116,5000
top,1,home,Three Stars,"These are AWFUL. They are see through, the fab..."
freq,1000,440,110,1


In [27]:
df.to_csv('csv/dataset_en_test.csv', index=False)

In [28]:
df.head()

,stars,category,title,body
0,1,apparel,Don’t waste your time!,"These are AWFUL. They are see through, the fab..."
1,1,other,One Star,I bought 4 and NONE of them worked. Yes I used...
2,1,other,Totally useless,On first use it didn't heat up and now it does...
3,1,jewelry,Gold filled earrings,You want an HONEST answer? I just returned fro...
4,1,industrial_supplies,Poor container,The glue works fine but the container is impos...


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   stars     5000 non-null   object
 1   category  5000 non-null   object
 2   title     5000 non-null   object
 3   body      5000 non-null   object
dtypes: object(4)
memory usage: 156.4+ KB
